In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
import scienceplots


In [ ]:
plt.style.use(['science', 'ieee'])  # Enables LaTeX + clean scientific styling

# LaTeX font settings
plt.rcParams.update({
    'text.usetex': False,
    'font.family': 'serif',
    'font.size': 8,
    'axes.titlesize': 9,
    'axes.labelsize': 8,
    'legend.fontsize': 7,
    'xtick.labelsize': 7,
    'ytick.labelsize': 7
})

In [ ]:
# Read CSV files
df_completion_tokens = pd.read_csv('avg_completion_tokens.csv')
df_prompt_tokens = pd.read_csv('avg_prompt_tokens.csv')
df_completion_per_agent = pd.read_csv('avg_completion_tokens_per_agent.csv')
df_prompt_per_agent = pd.read_csv('mean_prompt_tokens_per_agent.csv')

In [ ]:
# Display first few rows of avg_completion_token_usage
print("Average Completion Token Usage:")
print(df_completion_tokens.head())

In [ ]:
# Filter out MIN/MAX columns and step columns, keeping only the main metrics
def filter_columns(df):
    return df[[col for col in df.columns if all(x not in col for x in ['MIN', 'MAX', 'step'])]]

df_completion_tokens_filtered = filter_columns(df_completion_tokens)
df_prompt_tokens_filtered = filter_columns(df_prompt_tokens)
df_completion_per_agent_filtered = filter_columns(df_completion_per_agent)
df_prompt_per_agent_filtered = filter_columns(df_prompt_per_agent)

print("Filtered columns for completion tokens:")
print(df_completion_tokens_filtered.columns.tolist())

# Agent count and communication computation functions
import math
import re

def compute_num_agents(sequence_length, branching_factor):
    """
    Compute total number of agents in PrefixSumAgents hierarchical structure.
    
    Args:
        sequence_length: Length of input sequence (e.g., 32, 64, 128)
        branching_factor: Branching factor b (number of inputs each manager processes)
    
    Returns:
        int: Total number of manager agents needed
    """
    if branching_factor <= 1:
        return sequence_length  # Each element needs its own agent
    
    return math.ceil((sequence_length - 1) / (branching_factor - 1))

def compute_total_communication(sequence_length, branching_factor):
    """
    Compute total communication (number of edges) in PrefixSumAgents b-ary tree.
    
    Args:
        sequence_length: Length of input sequence (e.g., 32, 64, 128)  
        branching_factor: Branching factor b (number of inputs each manager processes)
    
    Returns:
        int: Total number of edges (communications) in the tree
    """
    if branching_factor <= 1:
        return sequence_length - 1  # Linear chain
    
    total_edges = 0
    current_level_size = sequence_length
    
    # Simulate the hierarchical processing
    while current_level_size > 1:
        # Number of managers at next level
        next_level_size = math.ceil(current_level_size / branching_factor)
        
        # Each manager receives up to b inputs, but we need to count actual edges
        # The number of edges from current level to next level equals current_level_size
        total_edges += current_level_size
        
        current_level_size = next_level_size
    
    return total_edges

def extract_sequence_info(column_name):
    """Extract sequence length from column name."""
    # Example: 'pareto_prefixsum_seq128_b2-64 - avg_completion_tokens'
    seq_match = re.search(r'seq(\d+)', column_name)
    sequence_length = int(seq_match.group(1)) if seq_match else None
    return sequence_length

In [ ]:
# Plotting function with color gradient, legends, and smoothing option
def plot_token_metrics(df, title, ylabel, smooth=False):
    plt.figure(figsize=(3.5, 2.2))  # Paper-friendly size matching other notebooks
    
    # Get branching factor (x-axis)
    x_values = df['branching_factor']
    
    # Get metric columns (excluding branching_factor)
    metric_cols = [col for col in df.columns if col != 'branching_factor']
    
    # Create color gradient using colors similar to the other notebooks
    colors = ['#4C72B0', '#55A868', '#C44E52']
    if len(metric_cols) > 3:
        colors = plt.cm.viridis(np.linspace(0, 1, len(metric_cols)))
    
    # Plot each metric
    for i, col in enumerate(metric_cols):
        # Extract a cleaner label from the column name
        label = col.replace('pareto_prefixsum_', '').replace(' - avg_completion_tokens', '').replace(' - avg_prompt_tokens', '').replace(' - mean_completion_tokens_per_agent', '').replace(' - mean_prompt_tokens_per_agent', '')
        
        y_values = df[col]
        color = colors[i % len(colors)]
        
        if smooth and len(x_values) > 3:
            # Create smooth interpolation
            x_smooth = np.linspace(x_values.min(), x_values.max(), 100)
            try:
                spl = make_interp_spline(x_values, y_values, k=min(3, len(x_values)-1))
                y_smooth = spl(x_smooth)
                plt.plot(x_smooth, y_smooth, color=color, linewidth=1.5, linestyle='-', label=label)
                # Add original data points
                plt.scatter(x_values, y_values, color=color, s=15, zorder=5)
            except:
                # Fall back to regular plot if smoothing fails
                plt.plot(x_values, y_values, 
                        marker='o', 
                        linewidth=1.5, 
                        markersize=3,
                        color=color,
                        linestyle='-',
                        label=label)
        else:
            plt.plot(x_values, y_values, 
                    marker='o', 
                    linewidth=1.5, 
                    markersize=3,
                    color=color,
                    linestyle='-',
                    label=label)
    
    plt.xlabel(r'\textbf{Branching Factor}')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend(frameon=False, loc='best')
    plt.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()

# Agent count plotting function
def plot_agents_vs_tokens(df_completion_tokens, title="Number of Agents vs Average Completion Tokens"):
    """Plot number of agents vs completion tokens for different sequence lengths."""
    
    plt.figure(figsize=(3.5, 2.2))
    
    # Extract unique sequence lengths from column names
    seq_lengths = set()
    for col in df_completion_tokens.columns:
        if col != 'branching_factor':
            seq_len = extract_sequence_info(col)
            if seq_len:
                seq_lengths.add(seq_len)
    
    colors = ['#4C72B0', '#55A868', '#C44E52']
    
    for i, seq_len in enumerate(sorted(seq_lengths)):
        # Find the column for this sequence length
        token_col = None
        for col in df_completion_tokens.columns:
            if f'seq{seq_len}' in col and col != 'branching_factor':
                token_col = col
                break
        
        if token_col:
            # Compute agent counts for this sequence length
            agent_counts = df_completion_tokens['branching_factor'].apply(
                lambda b: compute_num_agents(seq_len, b)
            )
            
            plt.plot(agent_counts, df_completion_tokens[token_col], 
                    marker='o', 
                    linewidth=1.5, 
                    markersize=3,
                    color=colors[i % len(colors)],
                    linestyle='-',
                    label=f'seq{seq_len}')
    
    plt.xlabel(r'\textbf{Number of Agents}')
    plt.ylabel(r'\textbf{Average Completion Tokens}')
    plt.title(title)
    plt.legend(frameon=False, loc='best')
    plt.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()

# Communication plotting function  
def plot_communication_vs_tokens(df_completion_tokens, title="Total Communication vs Average Completion Tokens"):
    """Plot total communication (edges) vs completion tokens for different sequence lengths."""
    
    plt.figure(figsize=(3.5, 2.2))
    
    # Extract unique sequence lengths from column names
    seq_lengths = set()
    for col in df_completion_tokens.columns:
        if col != 'branching_factor':
            seq_len = extract_sequence_info(col)
            if seq_len:
                seq_lengths.add(seq_len)
    
    colors = ['#4C72B0', '#55A868', '#C44E52']
    
    for i, seq_len in enumerate(sorted(seq_lengths)):
        # Find the column for this sequence length
        token_col = None
        for col in df_completion_tokens.columns:
            if f'seq{seq_len}' in col and col != 'branching_factor':
                token_col = col
                break
        
        if token_col:
            # Compute total communication for this sequence length
            communication_counts = df_completion_tokens['branching_factor'].apply(
                lambda b: compute_total_communication(seq_len, b)
            )
            
            plt.plot(communication_counts, df_completion_tokens[token_col], 
                    marker='o', 
                    linewidth=1.5, 
                    markersize=3,
                    color=colors[i % len(colors)],
                    linestyle='-',
                    label=f'seq{seq_len}')
    
    plt.xlabel(r'\textbf{Total Communication (Edges)}')
    plt.ylabel(r'\textbf{Average Completion Tokens}')
    plt.title(title)
    plt.legend(frameon=False, loc='best')
    plt.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot 1: Average Completion Token Usage (with smoothing)
plot_token_metrics(df_completion_tokens_filtered, 
                   'Average Completion Token Usage by Branching Factor', 
                   r'\textbf{Average Completion Tokens}',
                   smooth=True)

In [ ]:
# Plot 2: Average Prompt Tokens (with smoothing)
plot_token_metrics(df_prompt_tokens_filtered, 
                   'Average Prompt Tokens by Branching Factor', 
                   r'\textbf{Average Prompt Tokens}',
                   smooth=True)

In [ ]:
# Plot 3: Mean Completion Tokens per Agent (with smoothing)
plot_token_metrics(df_completion_per_agent_filtered, 
                   'Mean Completion Tokens per Agent by Branching Factor', 
                   r'\textbf{Mean Completion Tokens per Agent}',
                   smooth=True)

In [ ]:
# Plot 4: Mean Prompt Tokens per Agent (with smoothing)
plot_token_metrics(df_prompt_per_agent_filtered, 
                   'Mean Prompt Tokens per Agent by Branching Factor', 
                   r'\textbf{Mean Prompt Tokens per Agent}',
                   smooth=True)

# NEW PLOT: Number of Agents vs Average Completion Tokens
plot_agents_vs_tokens(df_completion_tokens_filtered, 
                     'Number of Agents vs Average Completion Tokens')

# NEW PLOT: Total Communication vs Average Completion Tokens  
plot_communication_vs_tokens(df_completion_tokens_filtered,
                            'Total Communication vs Average Completion Tokens')

In [ ]:
def plot_all_metrics_subplot():
    fig, axes = plt.subplots(2, 2, figsize=(7, 5))
    fig.suptitle('Token Usage Metrics by Branching Factor', fontsize=10, fontweight='bold')
    
    datasets = [
        (df_completion_tokens_filtered, r'\textbf{Avg Completion Tokens}', 'Average Completion Token Usage'),
        (df_prompt_tokens_filtered, r'\textbf{Avg Prompt Tokens}', 'Average Prompt Tokens'),
        (df_completion_per_agent_filtered, r'\textbf{Mean Completion Tokens/Agent}', 'Mean Completion Tokens per Agent'),
        (df_prompt_per_agent_filtered, r'\textbf{Mean Prompt Tokens/Agent}', 'Mean Prompt Tokens per Agent')
    ]
    
    colors = ['#4C72B0', '#55A868', '#C44E52']
    
    for idx, (df, ylabel, title) in enumerate(datasets):
        ax = axes[idx // 2, idx % 2]
        
        # Get branching factor (x-axis)
        x_values = df['branching_factor']
        
        # Get metric columns (excluding branching_factor)
        metric_cols = [col for col in df.columns if col != 'branching_factor']
        
        # Plot each metric with smoothing
        for i, col in enumerate(metric_cols):
            # Extract a cleaner label from the column name
            label = col.replace('pareto_prefixsum_', '').replace(' - avg_completion_tokens', '').replace(' - avg_prompt_tokens', '').replace(' - mean_completion_tokens_per_agent', '').replace(' - mean_prompt_tokens_per_agent', '')
            
            y_values = df[col]
            color = colors[i % len(colors)]
            
            # Apply smoothing
            if len(x_values) > 3:
                x_smooth = np.linspace(x_values.min(), x_values.max(), 100)
                try:
                    spl = make_interp_spline(x_values, y_values, k=min(3, len(x_values)-1))
                    y_smooth = spl(x_smooth)
                    ax.plot(x_smooth, y_smooth, color=color, linewidth=1.5, label=label, linestyle='-')
                    # Add original data points
                    ax.scatter(x_values, y_values, color=color, s=10, zorder=5, linestyle='-')
                except:
                    # Fall back to regular plot if smoothing fails
                    ax.plot(x_values, y_values, 
                            marker='o', 
                            linewidth=1.5, 
                            markersize=2,
                            color=color,
                            label=label,
                            linestyle='-')
            else:
                ax.plot(x_values, y_values, 
                        marker='o', 
                        linewidth=1.5, 
                        markersize=2,
                        color=color,
                        label=label)
        
        ax.set_xlabel(r'\textbf{Branching Factor}')
        ax.set_ylabel(ylabel)
        ax.set_title(title, fontsize=8)
        ax.legend(frameon=False, loc='best', fontsize=6)
        ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    
    plt.tight_layout()
    plt.show()

plot_all_metrics_subplot()

In [ ]:
# 4-Subplot Figure: Token Analysis vs Different X-Axes

def plot_completion_tokens_analysis():
    """Plot avg_completion_tokens and mean_completion_tokens_per_agent as function of branching factor, number of agents, and total communication."""
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    
    # Extract unique sequence lengths from column names
    seq_lengths = set()
    for col in df_completion_tokens_filtered.columns:
        if col != 'branching_factor':
            seq_len = extract_sequence_info(col)
            if seq_len:
                seq_lengths.add(seq_len)
    
    seq_lengths = sorted(seq_lengths)
    
    # Create colorscheme for arbitrary number of lines using viridis
    colors = plt.cm.viridis(np.linspace(0, 0.8, len(seq_lengths)))
    
    # Create color mapping for consistent colors across subplots
    color_map = {seq_len: colors[i] for i, seq_len in enumerate(seq_lengths)}
    
    # Plot 1: Branching Factor vs Avg Completion Tokens
    ax1 = axes[0, 0]
    x_values = df_completion_tokens_filtered['branching_factor']
    
    # Plot in order of sorted sequence lengths for consistent legend order
    for seq_len in seq_lengths:
        # Find the column for this sequence length
        token_col = None
        for col in df_completion_tokens_filtered.columns:
            if f'seq{seq_len}' in col and col != 'branching_factor':
                token_col = col
                break
        
        if token_col:
            y_values = df_completion_tokens_filtered[token_col]
            color = color_map[seq_len]
            
            # Filter out NaN values for plotting
            valid_mask = ~pd.isna(y_values)
            x_valid = x_values[valid_mask]
            y_valid = y_values[valid_mask]
            
            # Only plot if we have valid data points
            if len(x_valid) > 0:
                ax1.plot(x_valid, y_valid, 
                        marker='o', 
                        linewidth=1.5, 
                        markersize=3,
                        color=color,
                        linestyle='-',
                        label=f'N={seq_len}')
    
    ax1.set_xlabel(r'\textbf{Branching Factor}')
    ax1.set_ylabel(r'\textbf{Computation Depth}')
    ax1.legend(frameon=True, loc='best', fontsize=8)
    ax1.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    
    # Plot 2: Number of Agents vs Avg Completion Tokens
    ax2 = axes[0, 1]
    for seq_len in seq_lengths:
        # Find the column for this sequence length
        token_col = None
        for col in df_completion_tokens_filtered.columns:
            if f'seq{seq_len}' in col and col != 'branching_factor':
                token_col = col
                break
        
        if token_col:
            # Filter out NaN values first
            valid_mask = ~pd.isna(df_completion_tokens_filtered[token_col])
            valid_data = df_completion_tokens_filtered[valid_mask]
            
            if len(valid_data) > 0:
                # Compute agent counts for this sequence length (only for valid data)
                agent_counts = valid_data['branching_factor'].apply(
                    lambda b: compute_num_agents(seq_len, b)
                )
                
                ax2.plot(agent_counts, valid_data[token_col], 
                        marker='o', 
                        linewidth=1.5, 
                        markersize=3,
                        color=color_map[seq_len],
                        linestyle='-',
                        label=f'N={seq_len}')
    
    ax2.set_xlabel(r'\textbf{Number of Agents}')
    ax2.set_ylabel(r'\textbf{Computation Depth}')
    ax2.legend(frameon=True, loc='best', fontsize=8)
    ax2.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    
    # Plot 3: Total Communication vs Avg Completion Tokens
    ax3 = axes[1, 0]
    for seq_len in seq_lengths:
        # Find the column for this sequence length
        token_col = None
        for col in df_completion_tokens_filtered.columns:
            if f'seq{seq_len}' in col and col != 'branching_factor':
                token_col = col
                break
        
        if token_col:
            # Filter out NaN values first
            valid_mask = ~pd.isna(df_completion_tokens_filtered[token_col])
            valid_data = df_completion_tokens_filtered[valid_mask]
            
            if len(valid_data) > 0:
                # Compute total communication for this sequence length (only for valid data)
                communication_counts = valid_data['branching_factor'].apply(
                    lambda b: compute_total_communication(seq_len, b)
                )
                
                ax3.plot(communication_counts, valid_data[token_col], 
                        marker='o', 
                        linewidth=1.5, 
                        markersize=3,
                        color=color_map[seq_len],
                        linestyle='-',
                        label=f'N={seq_len}')
    
    ax3.set_xlabel(r'\textbf{Total Communication (Edges)}')
    ax3.set_ylabel(r'\textbf{Computation Depth}')
    ax3.legend(frameon=True, loc='best', fontsize=8)
    ax3.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    
    # Plot 4: Number of Agents vs Mean Completion Tokens per Agent
    ax4 = axes[1, 1]
    for seq_len in seq_lengths:
        # Find the column for this sequence length in the per-agent dataframe
        token_col = None
        for col in df_completion_per_agent_filtered.columns:
            if f'seq{seq_len}' in col and col != 'branching_factor':
                token_col = col
                break
        
        if token_col:
            # Filter out NaN values first
            valid_mask = ~pd.isna(df_completion_per_agent_filtered[token_col])
            valid_data = df_completion_per_agent_filtered[valid_mask]
            
            if len(valid_data) > 0:
                # Compute agent counts for this sequence length (only for valid data)
                agent_counts = valid_data['branching_factor'].apply(
                    lambda b: compute_num_agents(seq_len, b)
                )
                
                ax4.plot(agent_counts, valid_data[token_col], 
                        marker='o', 
                        linewidth=1.5, 
                        markersize=3,
                        color=color_map[seq_len],
                        linestyle='-',
                        label=f'N={seq_len}')
    
    ax4.set_xlabel(r'\textbf{Number of Agents}')
    ax4.set_ylabel(r'\textbf{Average CoT Length}')
    ax4.legend(frameon=True, loc='best', fontsize=8)
    ax4.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    
    plt.tight_layout()
    plt.show()

plot_completion_tokens_analysis()

# Display some example calculations for verification
print("Example calculations for sequence length 32:")
print("Branching Factor | Num Agents | Total Communication")
print("-----------------|------------|-------------------")
for b in [2, 4, 6, 8, 10, 12, 14, 16, 18]:
    agents = compute_num_agents(32, b)
    comm = compute_total_communication(32, b)
    print(f"       {b:2d}        |     {agents:2d}     |        {comm:3d}")

In [ ]:
# Total Computation vs Total Communication Analysis

def plot_computation_vs_communication():
    """Plot Total Computation (tokens) vs Total Communication (edges) using style similar to permutations notebook."""
    
    plt.figure(figsize=(3.5, 2.5))
    
    # Extract unique sequence lengths from column names
    seq_lengths = set()
    for col in df_completion_tokens_filtered.columns:
        if col != 'branching_factor':
            seq_len = extract_sequence_info(col)
            if seq_len:
                seq_lengths.add(seq_len)
    
    seq_lengths = sorted(seq_lengths)
    # Use 4 colors for the 4 sequence lengths
    colors = ['#4C72B0', '#55A868', '#C44E52', '#DD8452']
    
    for i, seq_len in enumerate(seq_lengths):
        # Find the column for this sequence length
        token_col = None
        for col in df_completion_tokens_filtered.columns:
            if f'seq{seq_len}' in col and col != 'branching_factor':
                token_col = col
                break
        
        if token_col:
            # Filter out NaN values first
            valid_mask = ~pd.isna(df_completion_tokens_filtered[token_col])
            valid_data = df_completion_tokens_filtered[valid_mask]
            
            if len(valid_data) > 0:
                # Compute total communication (edges)
                total_communications = valid_data['branching_factor'].apply(
                    lambda b: compute_total_communication(seq_len, b)
                )
                
                # Use completion tokens as total computation
                total_computation = valid_data[token_col]
                
                plt.plot(total_communications, total_computation, 
                        marker='o', 
                        linewidth=1.5, 
                        markersize=4,
                        color=colors[i % len(colors)],
                        linestyle='-',
                        label=f'N={seq_len}')
    
    plt.xlabel(r'\textbf{Total Communication (Edges)}')
    plt.ylabel(r'\textbf{Total Computation (Tokens)}')
    plt.title(r'\textbf{Total Computation vs Total Communication}')
    plt.legend(frameon=False, loc='upper right')
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    plt.tight_layout()
    plt.savefig("computation_vs_communication.pdf", bbox_inches='tight')
    plt.show()

def plot_computation_vs_communication_powers_of_2():
    """Plot Total Computation (tokens) vs Total Communication (edges) for powers of 2 branching factors only."""
    
    plt.figure(figsize=(3.5, 2.5))
    
    # Extract unique sequence lengths from column names
    seq_lengths = set()
    for col in df_completion_tokens_filtered.columns:
        if col != 'branching_factor':
            seq_len = extract_sequence_info(col)
            if seq_len:
                seq_lengths.add(seq_len)
    
    seq_lengths = sorted(seq_lengths)
    # Use 4 colors for the 4 sequence lengths
    colors = ['#4C72B0', '#55A868', '#C44E52', '#DD8452']
    
    for i, seq_len in enumerate(seq_lengths):
        # Find the column for this sequence length
        token_col = None
        for col in df_completion_tokens_filtered.columns:
            if f'seq{seq_len}' in col and col != 'branching_factor':
                token_col = col
                break
        
        if token_col:
            # Filter for powers of 2 branching factors and NaN values
            powers_of_2_mask = df_completion_tokens_filtered['branching_factor'].apply(
                lambda b: b > 0 and (b & (b - 1)) == 0  # Check if b is a power of 2
            )
            valid_mask = ~pd.isna(df_completion_tokens_filtered[token_col]) & powers_of_2_mask
            valid_data = df_completion_tokens_filtered[valid_mask]
            
            if len(valid_data) > 0:
                # Compute total communication (edges)
                total_communications = valid_data['branching_factor'].apply(
                    lambda b: compute_total_communication(seq_len, b)
                )
                
                # Use completion tokens as total computation
                total_computation = valid_data[token_col]
                
                plt.plot(total_communications, total_computation, 
                        marker='o', 
                        linewidth=1.5, 
                        markersize=4,
                        color=colors[i % len(colors)],
                        linestyle='-',
                        label=f'N={seq_len}')
    
    plt.xlabel(r'\textbf{Total Communication (Edges)}')
    plt.ylabel(r'\textbf{Total Computation (Tokens)}')
    #plt.title(r'\textbf{Total Computation vs Total Communication (Powers of 2)}')
    plt.legend(frameon=False, loc='upper right')
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    plt.tight_layout()
    plt.savefig("computation_vs_communication_powers_of_2.pdf", bbox_inches='tight')
    plt.show()

plot_computation_vs_communication()
plot_computation_vs_communication_powers_of_2()

# Display example calculations for verification
print("\nExample calculations for sequence length 64 (all branching factors):")
print("Branching Factor | Total Communication (Edges) | Avg Completion Tokens")
print("-----------------|------------------------------|----------------------")
seq64_col = None
for col in df_completion_tokens_filtered.columns:
    if 'seq64' in col and col != 'branching_factor':
        seq64_col = col
        break

if seq64_col:
    for b in [2, 4, 6, 8, 10, 12, 14, 16]:
        comm = compute_total_communication(64, b)
        tokens_row = df_completion_tokens_filtered[df_completion_tokens_filtered['branching_factor'] == b]
        if not tokens_row.empty:
            tokens = tokens_row[seq64_col].iloc[0]
            if pd.notna(tokens):
                is_power_of_2 = b > 0 and (b & (b - 1)) == 0
                marker = " *" if is_power_of_2 else ""
                print(f"       {b:2d}        |             {comm:3d}              |        {tokens:.1f}{marker}")

print("\n* = Power of 2 (included in second plot)")

print("\nPowers of 2 branching factors:", [b for b in df_completion_tokens_filtered['branching_factor'] if b > 0 and (b & (b - 1)) == 0])